In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(phyloseq)
library(ggplot2)
library(foreach)
library(doParallel)
library(gridExtra)
library(vegan)
library(plyr); library(dplyr)
library(RColorBrewer)
library(stats)
library(reshape2)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: foreach: simple, scalable parallel programming from Revolution Analytics
Use Revolution R for scalability, fault tolerance and more.
http://www.revolutionanalytics.com

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: iterators

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: parallel

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: permute

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: lattic

## Filtering fasta file used as OTU seeds by OTUs that are in the final l2fc dataset

In [4]:
%%R
l2fc = read.csv("data/l2fc_UNITE.csv", header = TRUE, sep = ",")

In [5]:
%%R
head((unique(l2fc$OTU)))
a = as.matrix((unique(l2fc$OTU)))
head(a)

     [,1]     
[1,] "OTU.2"  
[2,] "OTU.22" 
[3,] "OTU.19" 
[4,] "OTU.5"  
[5,] "OTU.308"
[6,] "OTU.3"  


In [8]:
%%R
write(a, file = "data/l2fc_OTUs.txt",
      ncolumns = 1,
      append = FALSE, sep = " ")

In [9]:
! head data/l2fc_OTUs.txt 

OTU.2
OTU.22
OTU.19
OTU.5
OTU.308
OTU.3
OTU.151
OTU.7
OTU.1081
OTU.8


In [11]:
! head SIP_Bulk_otusn_soft.fasta

>OTU.1
TTACAGAGTCGCAATACTCCGTAAACCATTGTGAACGTTACCTAACCGTTGCTTCGGCGGGCGGCGCCCGGGCCCTCGCGGCCCGCGGCGCCccccGGCCCCTGCGGGCGCCCGCCGGAGGTAGACCAAACTCTTGAATTACATGGCCTctctGAGTCTTCTGTACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>OTU.2
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTTCAAACCGTTGCTTCGGCGGGCGGCCCGGGTCCGCCCGGTGCCcccTGGCCcccTAGCGGGGCGCCCGCCGGAGGAAACCCAACTCTTGATTATTATGGCCTctctGAGTCTTCTGTACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>OTU.3
TTACTGAGTACTACACTctctACCCTTTGTGAACTATTATACCTGTTGCTTCGGCGGCGCCCGCGAGGGTGCCCGCCGGTCTCATCAGAATCTCTGTTTTCGAACCCGACGATACTTCTGAGTGTTCTTAGCGAACTGTCAAAACTTTTAACAACGGATCTCTTGGCTCCAGC
>OTU.4
TTACAGAGTCGCAACACTCCGTAAACCATTGTGAACGTTACCTTAACCGTTGCTTCGGCGGGCGGCGCCCGGGCCTCACCGCCCGCGGCGCCccccGGCCccccGTGGGCGCCCGCCGGAGGTACACCAAACTCTTGCATTGTATGGCCTctctGAGTATTCTGTACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>OTU.5
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCAAAACCGTTGCTTCGGCGGGCGGCCCGGGTCCTTCCCGGCGCCccccGGCCCTCGCGGGCGCCCGCCGGAGGTAAACCAAACTATTGCATTAATGGCCTctctGAGTCTTCTGTACTGAATA

In [ ]:
! grep -A 1 -wf data/l2fc_OTUs.txt SIP_Bulk_otusn_soft.fasta > data/SIP_Bulk_otusn_l2fc.fasta #or reg exp ...doing partial match

In [ ]:
##invert grep for lines with --

In [ ]:
! head data/SIP_Bulk_otusn_l2fc.fasta


In [ ]:
! grep -c "OTU" data/SIP_Bulk_otusn_l2fc.fasta


In [ ]:
! grep -c "OTU" data/l2fc_OTUs.txt


## Running Pairwise Comparisons on Filtered fasta file

In [ ]:
%%bash
ionice -c2 -n3 \
mothur "#pairwise.seqs(fasta=data/SIP_Bulk_otusn_l2fc.fasta, calc=onegap, countends=false, \
processors=10, output=square)" #> /dev/null


In [ ]:
!tail data/SIP_Bulk_otusn_l2fc.square.dist | cut -c1-80

In [ ]:
! grep -c "OTU" data/SIP_Bulk_otusn_l2fc.square.dist

## Running t-SNE analysis on pairwise distance matrix

In [ ]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from math import log

In [ ]:
%matplotlib inline

In [ ]:
dist_lines = open("data/SIP_Bulk_otusn_l2fc.square.dist").readlines()
def line2array(line):
    line = line.rstrip()
    return np.array(map(float, line.split("\t")[1:]))
distmat = np.vstack([line2array(line) for line in dist_lines[1:]])

In [ ]:
def getOTU(line):
    line = line.rstrip()
    return np.array(line.split("\t")[0:1])
OTU= np.vstack(getOTU(line) for line in dist_lines[1:])

In [ ]:
print OTU

In [ ]:
print distmat

In [ ]:
def corr(a):
    if (a >= 0.75): return a
    else: return (-3/4*(log(1-(4/3*a))))

In [ ]:
vecfunc = np.vectorize(corr)
result=vecfunc(distmat)

In [ ]:
print result

In [ ]:
model = TSNE(n_components=2, random_state=0)
tsne_coords = model.fit_transform(result) 

In [ ]:
tsne_coords

In [ ]:
combo = np.concatenate((tsne_coords, OTU), axis = 1)

In [ ]:
combo

## Visualizing t-SNE coordinates

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(combo[:, 0], combo[:, 1])

## Importing python variables of t-SNE coordinates into R

In [ ]:
%Rpush combo

In [ ]:
%%R
colnames(combo) = c("tsne_1", "tsne_2", "OTU")

In [ ]:
%%R
combo = as.data.frame(combo)

In [ ]:
%%R
head(combo)

In [ ]:
%%R
combo$OTU = gsub(" ", "", combo$OTU, fixed = TRUE)

In [ ]:
%%R
head(combo)

## Creating csv with t-SNE information

In [ ]:
%%R
write.table(combo, "data/tSNE.csv", row.names = FALSE, sep = ",")

# Start notebook here if just visualizing t-SNE

In [ ]:
%%R
combo = read.csv("data/tSNE.csv")

## Adding Metadata to t-SNE coordinates

In [ ]:
%%R
sample.md = read.table("data/Metadata_fungi.txt", 
                comment.char = "",
                header = TRUE,
                colClasses = c("Density" = "numeric"))
rownames(sample.md) = as.character(sample.md[, 1])

sample.md = sample_data(sample.md)

In [ ]:
%%R
physeq = import_biom("data/otu_table_wtax.biom")
print(physeq)
sample.data = import_qiime_sample_data("data/Metadata_fungi.txt")
physeq = merge_phyloseq(physeq,sample.data)
print(physeq)
physeq.md = sample_data(physeq)
physeq.fungi = prune_samples((physeq.md$Treatment %in% c("12CCPS","13CCPS"))& 
                             (physeq.md$SampleID != c("D312.6")),  physeq)
physeq.fungi.prune = prune_samples(sample_sums(physeq.fungi) > 5360, physeq.fungi)
print (physeq.fungi.prune)
physeq.fungi.snorm = transform_sample_counts(physeq.fungi.prune, function(x) x/sum(x))
#physeq.fungi.thresh = filter_taxa(physeq.fungi.snorm, function(x) sum(x > 0) / length(x) > (0.25), TRUE)
#physeq.fungi.thresh.min = filter_taxa(physeq.fungi.thresh, function(x) sum(x) > 5000, TRUE)
#physeq.fungi.thresh = filter_taxa(physeq.SIP.snorm, function(x) sum(x > 0) > (0.25 * length(x)), TRUE)
#print(physeq.fungi.thresh)

In [ ]:
%%R
mdf= tbl_df(psmelt(physeq.fungi.prune))

In [ ]:
%%R
d = left_join(combo, mdf)

In [ ]:
%%R
d$Rank2 = gsub("p__", "", d$Rank2)
d$Rank3 = gsub("c__", "", d$Rank3)
d$Rank4 = gsub("o__", "", d$Rank4)

In [ ]:
%%R
head(d$Rank1)

In [ ]:
%%R
d_sub = select(d, OTU, Sample, Day, tsne_1, tsne_2, Density, Rank1, Rank2, Rank3, Treatment)

In [ ]:
%%R -w 800 -h 800

asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(d_sub, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(aes(fill = as.factor(Day), color = as.factor(Day), alpha = 0.85), pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6)))


p      

In [ ]:
%%R
l2fc = read.csv("data/l2fc_UNITE.csv", header = TRUE, sep = ",")

In [ ]:
%%R
d2 = left_join(combo, l2fc)

In [ ]:
%%R -w 800 -h 800

my_palette <- colorRampPalette(c("red", "yellow", "green"))(n = 299)


asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(d2, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(aes(fill = log2FoldChange, color = log2FoldChange, alpha = 0.85), pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6))) + labs(size = "Density")+
    scale_fill_gradient(low="blue",high="green") + scale_colour_gradient(low="blue",high="green") 


p     

In [ ]:
%%R
head(d2)
d2$logp = -log10(d2$padj)

In [ ]:
%%R
d2$Resp_status[d2$padj > 0.10] = "Non-responder"
d2$Resp_status[d2$padj <= 0.10] = "Responder"
d2$Resp_status[is.na(d2$padj)] = "Non-responder"

## Visualizing t-SNE Coordinates with Metadata painted on

In [ ]:
%%R -w 800 -h 600

my_palette <- colorRampPalette(c("red", "yellow", "green"))(n = 299)


asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(d2, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(fill = "blue", color = "blue", size = 2, pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6)))  + 
    #scale_colour_gradient(limits = c(0, 1), low="red", high="blue", name = "Adjusted p-value") +
    guides(color=FALSE) + theme(text = element_text(size=18)) 


p = p + xlab("tsne1") + ylab("tsne2")


p   

In [ ]:
%%R
ggsave("data/figs/tnseFig_nolabels.png", p, height = 7, width = 7)

In [ ]:
%%R -w 800 -h 600

my_palette <- c("blue", "red", "grey")


asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(d2, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(aes(fill = Resp_status, color = Resp_status, alpha = 0.85), size = 2.5, pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=5))) +
    scale_shape_discrete(name = "Responder Status") + 
    scale_colour_manual(values = my_palette) +
    scale_fill_manual(values = my_palette, name = "Responder Status")+
    guides(color=FALSE) + theme(text = element_text(size=18)) 

p = p + geom_point(data = subset(d2, padj <= 0.10), shape = 20, size = 3, color = "red")

p = p + theme(legend.position ="bottom")

p = p + xlab("tsne1") + ylab("tsne2")

p = p + guides(fill = FALSE)


p     

In [ ]:
%%R
ggsave("data/figs/tnseFig.png", p, height = 8, width = 8)

In [ ]:
%%R
d3 = d2 %>% group_by(tsne_1, tsne_2, padj, OTU) %>%
    summarise(LFC = mean(log2FoldChange))

In [ ]:
%%R
points.dist = dist(cbind(d3$tsne_1, d3$tsne_2))
pval.dist = dist(d3$padj)

In [ ]:
%%R
points2.dist = as.matrix(points.dist)
pval2.dist = as.matrix(pval.dist)

In [ ]:
%%R
library(ape)

In [ ]:
%%R

na = filter(d2, is.na(Rank3))
rest = filter(d2, !is.na(Rank3))
na$group = "Unclassified"
rest$group = rest$Rank3
g = rbind(na, rest)

In [ ]:
%%R
head(g)

In [ ]:
%%R -w 800 -h 600

#my_palette <- colorRampPalette(c("red", "yellow", "green"))(n = 299)


asNumeric = function(x) { as.numeric(as.character(x)) }
p2 = ggplot(g, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))
#p = p + geom_point(data = subset(d2, padj <= 0.10), shape = 20, size = 6)
p2 = p2 + geom_point(aes(fill = group, color = group, alpha = 0.85), pch = 21, size = 2.5) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=5))) +
    guides(colour=FALSE)

colourCount = length(unique(g$group)) + 1
getPalette = colorRampPalette(brewer.pal(10, "Set1"))
p2 = p2 + scale_color_manual(values = getPalette(colourCount))
p2 = p2 + scale_fill_manual(values = getPalette(colourCount), name = "Class")+ theme(text = element_text(size=18)) 

#p = p  + geom_point(data = subset(d2, Rank3 %in% c('unclassified', 'unidentified')), color = "black", size = 2)
p2 = p2 + theme(legend.position= "bottom") + guides(fill=guide_legend(ncol=6))
#p2 = p2 + guides(fill = FALSE)




p2 = p2 + xlab("tsne1") + ylab("tsne2") +  guides(color=FALSE) 


p2   

In [ ]:
%%R
tiff("data/figs/tnse.combo.tiff", res = 75, )
grid.arrange(p, p2, ncol=2)
dev.off()

In [ ]:
%%R -w 800 -h 600

#my_palette <- colorRampPalette(c("red", "yellow", "green"))(n = 299)


asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(g, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))
p = p + geom_point(data = subset(d2, padj <= 0.10), shape = 20, size = 7)
p = p + geom_point(aes(fill = group, color = group), pch = 21, size = 2.5) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6))) +
    #scale_fill_gradient(limits = c(0, 1), low="blue",high="green") + 
    #scale_colour_gradient(limits = c(0, 1), low="blue", high="green", name = "adjusted p-value") +
    guides(colour=FALSE)

colourCount = length(unique(d2$Rank3)) + 1
getPalette = colorRampPalette(brewer.pal(10, "Set1"))
p = p + scale_color_manual(values = getPalette(colourCount))
p = p + scale_fill_manual(values = getPalette(colourCount), name = "Class")+ theme(text = element_text(size=18)) 

#p = p  + geom_point(data = subset(d2, Rank3 %in% c('unclassified', 'unidentified')), color = "black", size = 2)




p = p + xlab("tsne1") + ylab("tsne2") +  guides(color=FALSE) 


p     

In [ ]:
%%R -w 800 -h 600
resp = filter(d2, padj <= 0.10)

asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(resp, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(aes(fill = as.factor(Day), color = as.factor(Day), alpha = 0.85), pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6))) +
    #scale_fill_gradient(limits = c(0, 1), low="blue",high="green") + 
    #scale_colour_gradient(limits = c(0, 1), low="blue", high="green", name = "adjusted p-value") +
    guides(fill=FALSE)

p = p + xlab("tsne1") + ylab("tsne2")


p     


In [ ]:
%%R
ggsave("data/figs/tnseFig.jpg", p, height = 5, width = 7)

In [ ]:
%%R -w 800 -h 300

dcounts = d2 %>%
          group_by(Day,Rank2) %>%
          summarize(responders = sum(padj<= 0.10)) 

dcounts = filter(dcounts, responders !=0)          
dcounts$Rank2[is.na(dcounts$Rank2)] = "p__unidentified"

p2 = ggplot(dcounts, aes(x = Rank2 , y = responders, fill = Rank2)) +
    facet_grid(. ~ Day, scales = "free_x", space = "free") +
    geom_bar(stat = "identity") +
    xlab("Phyla") + ylab("responder count") +
    scale_x_discrete(breaks=NULL) +
    scale_fill_discrete(name = "Phyla", labels = c("Ascomycota", "Basidiomycota", "Unidentified"))
                        

p2

In [ ]:
%%R
ggsave("data/figs/responderCounts.jpg", p2, height = 3, width = 8)

In [ ]:
%%R
tail(filter(d2, Rank2 == "p__unidentified"))

In [ ]:
%%R -w 1000 -h 300

dcounts = d2 %>%
          group_by(Day,Rank3) %>%
          summarize(responders = sum(padj<= 0.10)) 

dcounts = filter(dcounts, responders !=0)          


p = ggplot(dcounts, aes(x = Rank3 , y = responders, fill = Rank3)) +
    facet_grid(. ~ Day, scales = "free_x", space = "free") +
    geom_bar(stat = "identity")

p

In [ ]:
%%R
head(dcounts)

In [ ]:
%%R
d3 = filter(d2, padj <=0.10)
d3$tsne_1 = asNumeric(d3$tsne_1)
d3$tsne_2 = asNumeric(d3$tsne_2)
d3 = filter(d3, tsne_1 > 19)
d3 = filter(d3, tsne_2 > 7)
d3 = filter(d3, tsne_2 <13)

In [ ]:
%%R
length(d3$OTU)

In [ ]:
%%R -w 600 -h 600

my_palette <- colorRampPalette(c("red", "yellow", "green"))(n =299)


asNumeric = function(x) { as.numeric(as.character(x)) }
p = ggplot(d3, aes(x = asNumeric(tsne_1), y = asNumeric(tsne_2)))+
    geom_point(aes(fill = padj, color = padj, alpha = 0.85), pch = 21) +
    guides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6))) +
    scale_fill_gradient(low="red",high="yellow") + scale_colour_gradient(low="red",high="yellow") +
    guides(fill=FALSE)


p     

In [ ]:
%%R
OTUs = d3$OTU 

In [ ]:
%%R
counts = filter(d, OTU %in% OTUs)

In [ ]:
%%R
head(dcounts)

In [ ]:
%%R

dcounts = d3 %>%
          group_by(Day,Rank6) %>%
          summarize(responders = sum(padj<= 0.10))

p = ggplot(dcounts, aes(x = Rank6 , y = responders, fill = Rank6)) +
    facet_grid(. ~ Day, scales = "free_x", space = "free") +
    geom_bar(stat = "identity")


#px = ggplot(d, aes(x = factor(Day), y = responders, fill = RNA_DNA))

#px = px + facet_grid(. ~ Day, scales = "free_x", space = "free")

#px = px + geom_bar(stat = "identity", position = "dodge")

    #geom_point(aes(fill = padj, color = padj, alpha = 0.85), pch = 21) +
    #uides(alpha = FALSE, fill=guide_legend(override.aes=list(size=6))) +
    #scale_fill_gradient(low="red",high="yellow") + scale_colour_gradient(low="red",high="yellow") +
    #guides(fill=FALSE)
p

In [ ]:
%%R
head(d3)

###Blasting vs. Living Tree Project for Responders

In [ ]:
%%R
resp = filter(d2, padj<=0.10)

In [ ]:
%%R
q = arrange(resp, padj)
OTU = unique(q$OTU)
length(OTU)

In [ ]:
%%R
head(OTU, n = 10)

In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline as blast

In [ ]:
from cogent import LoadSeqs, DNA

In [ ]:
%%bash
formatdb -i /home/chantal/FungalSIP/data/tmp/db/its_12_11_otus/rep_set/97_otus.fasta -p F

In [ ]:
OTU_ids = ["OTU.98", "OTU.49", "OTU.53", "OTU.90", "OTU.279", "OTU.6893", "OTU.460", "OTU.6893"]
seqs = LoadSeqs("/home/chantal/FungalSIP/data/otusn.fasta", moltype=DNA, aligned=False)
seqs = seqs.takeSeqs(OTU_ids).toFasta()
print seqs

In [ ]:
my_file = open("seqs.fasta", "w")
my_file.write(seqs)
my_file.close()

In [ ]:
cline = blast(query = "seqs.fasta",
              db = "/home/chantal/FungalSIP/data/tmp/db/its_12_11_otus/rep_set/97_otus.fasta",
              outfmt = 6,
              out = "otusn_silvaRefNR115.m6",
              num_threads = 5)
print cline
cline()

In [ ]:
!head -n1 otusn_silvaRefNR115.m6
!wc -l otusn_silvaRefNR115.m6
! grep -c "OTU.279" otusn_silvaRefNR115.m6

In [ ]:
from IPython.display import Image
import pandas.rpy.common as com

In [ ]:
%%R
library(data.table)
library(dplyr)

In [ ]:
%%R
blastout_dt = fread("otusn_silvaRefNR115.m6")
blastout_df = tbl_df(blastout_dt)
colnames(blastout_df) = c("OTU",
                        "acc",
                        "pid",
                        "aln.length",
                        "mismatches",
                        "gapOpenCount",
                        "qstart",
                        "qend",
                        "sstart",
                        "send",
                        "e.value",
                        "bit.score")

top_hits_df_LTP = blastout_df %>% 
                          group_by(OTU) %>% 
                          mutate(Rank = rank(desc(bit.score), ties.method = "min")) #%>% 
                          #filter(pid > 97.0) %>%
                          #filter(e.value < 5.0e-180) %>%
                          #filter(acc %in% c('AM286414', 'AB428568', 'EU877263', 'EF575565', 'EF494193', 'X82449'))

In [ ]:
%%R
print(head(top_hits_df_LTP))
str(top_hits_df_LTP$OTU)

In [ ]:
! head data/tmp/db/its_12_11_otus/taxonomy/otu_taxonomy.txt

In [ ]:
%%R
dt = fread("data/tmp/db/its_12_11_otus/taxonomy/97_otu_taxonomy.txt", sep = ";", header = FALSE)
dat1 <- data.frame(do.call(rbind, strsplit(as.vector(dt$V1), split = "\t")))
dt <- cbind(dat1, dt)
dt$V1 = NULL
colnames(dt) = c("acc","Kingdom","Phylum","Class","Order","Family","Genus", "Species")
ITS_seq_data = tbl_df(dt)
head(dt)

In [ ]:
%%R
joined_df = left_join(top_hits_df_LTP, ITS_seq_data)

In [ ]:
%%R
head(joined_df)

In [ ]:
%%R
joined_df = filter(joined_df, name != "NA")

In [ ]:
%%R
dim(joined_df)

In [ ]:
%%R
head(joined_df)

In [ ]:
df = com.load_data("joined_df")

In [ ]:
def get_latex(df):
    latex = ""
    prefix = r"\multirow{%s}{*}{%s}"%(len(df.index), df.OTU.iloc[0])
    for i, r in df[["Genus", "Species","pid","acc"]].iterrows():
        latex += " & " + " & ".join([str(i) for i in r.values]) + r" \\ "
    return prefix + latex + r"\midrule" + "\n"

In [ ]:
s = df.groupby("OTU").apply(get_latex)

table_framework = r"""
\documentclass[11pt]{article}
\usepackage{multirow, array, booktabs}
\begin{document}

\thispagestyle{empty}
 
\begin{table}[ht]
\caption{$^{13}$C RNA responders BLAST against Living Tree Project}
\begin{center}
\begin{tabular}{l>{\itshape}lrl}
    \toprule \\
    \textbf{OTU ID} & \textbf{Species Name} & \textbf{BLAST percent identity} & \textbf{accession} \\
    \midrule
    %s
    \bottomrule
\end{tabular}
\end{center}
\label{tab:multicol}
\end{table}
 
\end{document}"""%"".join(s.values)

with open("data/table.tex", "w") as out:
    out.write(table_framework)

In [ ]:
!latex -output-directory="data/tmp" data/tmp/table.tex >/dev/null 2>&1 && \
dvipng -o data/table.png data/tmp/table.dvi >/dev/null 2>&1
Image("data/table.png")